In [1]:
#Sickit learn met régulièrement à jour des versions et
#indique des futurs warnings.
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
# librairies générales
import pandas as pd
import re
from tabulate import tabulate
import time
import numpy as np
import pickle
import string
import base64
import sys
# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns
# librairies scikit learn
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
# librairies des classifiers utilisés
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
# librairies NLTK
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')
my_local_drive='/content/gdrive/My Drive/Colab Notebooks/TER'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive
%pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/TER


'/content/gdrive/My Drive/Colab Notebooks/TER'

In [3]:
import sys
sys.path.append("/content/gdrive/My Drive/Colab Notebooks/projet_ML")
from MyNLPUtilities import *

In [4]:
import pandas as pd

# Read the Excel file
df = pd.read_excel("all_data.xlsx", engine="openpyxl")

# Display the first few rows
print(df.head())

  Patient_ID Chip_Code Chip_Type Chip_Image_Name   Age Gender  \
0    XPW0007   XPW0007   ISAC_V1             NaN   9.0      M   
1    XPW0011   XPW0011   ISAC_V1             NaN  25.0      F   
2    XPW0013   XPW0013   ISAC_V1             NaN  59.0      F   
3    XPW0017   XPW0017   ISAC_V1             NaN  49.0      F   
4    XPW0018   XPW0018   ISAC_V1             NaN   9.0      F   

   Blood_Month_sample  French_Residence_Department  French_Region  \
0                 3.0                           32             11   
1                 5.0                           31             11   
2                 5.0                           82             11   
3                 5.0                           33             10   
4                 5.0                           33             10   

   Rural_or_urban_area  ...  Tyr_p_2  Ulm_c  Urt_d  Vac_m  Ves_v  Ves_v_1  \
0                    0  ...      NaN    NaN    NaN    NaN    NaN      NaN   
1                    1  ...      NaN    

In [5]:
# Filtrer les lignes où Chip_type == "ISAC_V1"
V1_df = df[df["Chip_Type"] == "ISAC_V1"]

#extraire to an excel sheet
#V1_df.to_excel("V1.xlsx", index=False)
V1_df.head()

,Patient_ID,Chip_Code,Chip_Type,Chip_Image_Name,Age,Gender,Blood_Month_sample,French_Residence_Department,French_Region,Rural_or_urban_area,...,Tyr_p_2,Ulm_c,Urt_d,Vac_m,Ves_v,Ves_v_1,Vit_v_1,Xip_g_1,Zea_m,Zea_m_14
0,XPW0007,XPW0007,ISAC_V1,NaN,9.0,M,3.0,32,11,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XPW0011,XPW0011,ISAC_V1,NaN,25.0,F,5.0,31,11,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,XPW0013,XPW0013,ISAC_V1,NaN,59.0,F,5.0,82,11,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,XPW0017,XPW0017,ISAC_V1,NaN,49.0,F,5.0,33,10,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,XPW0018,XPW0018,ISAC_V1,NaN,9.0,F,5.0,33,10,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

#repartition par age
fig1 = px.histogram(df, x="Age", nbins=20, color_discrete_sequence=["skyblue"],
                    title="Répartition des âges")
fig1.update_layout(xaxis_title="Âge", yaxis_title="Nombre de personnes")

# prevelement par mois
df_month = df["Blood_Month_sample"].dropna().astype(int).value_counts().sort_index().reset_index()
df_month.columns = ["Mois", "Nombre"]
fig2 = px.bar(df_month, x="Mois", y="Nombre", color_discrete_sequence=["lightcoral"],
              title="Nombre de prélèvements par mois")
fig2.update_layout(xaxis_title="Mois de prélèvement", yaxis_title="Nombre de personnes")

# region francaise
region_counts = df['French_Region'].value_counts().reset_index()
region_counts.columns = ["Région", "Nombre"]
fig4 = px.bar(region_counts, x="Région", y="Nombre",
              color_discrete_sequence=["#2ca02c"],  # Vert
              title="Répartition par région française")
fig4.update_layout(xaxis_title="Région française",
                  yaxis_title="Nombre de personnes",
                  xaxis_tickangle=-45)

fig = make_subplots(rows=1, cols=3, subplot_titles=("Répartition des âges", "Prélèvements par mois", "Répartition par région française"),
                   shared_yaxes=False)

# Ajouter les sous-graphiques
for trace in fig1["data"]:
    fig.add_trace(trace, row=1, col=1)

for trace in fig2["data"]:
    fig.add_trace(trace, row=1, col=2)

for trace in fig4["data"]:
    fig.add_trace(trace, row=1, col=3)

# Mettre en page
fig.update_layout(
    height=500,
    width=1200,
    showlegend=False,
    margin=dict(r=50, l=50, b=100),  # Ajuster les marges pour les labels
    # Couleurs uniques pour chaque graphique
    colorway=["#1f77b4", "#ff7f0e", "#2ca02c"]
)


# Rotation des étiquettes sur l'axe x pour la région
fig.update_xaxes(tickangle=-45, row=1, col=3)

fig.show()

In [26]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

#repartition par age
fig1 = px.histogram(V1_df, x="Age", nbins=20, color_discrete_sequence=["skyblue"],
                    title="Répartition des âges")
fig1.update_layout(xaxis_title="Âge", yaxis_title="Nombre de personnes")

# prevelement par mois
df_month = V1_df["Blood_Month_sample"].dropna().astype(int).value_counts().sort_index().reset_index()
df_month.columns = ["Mois", "Nombre"]
fig2 = px.bar(df_month, x="Mois", y="Nombre", color_discrete_sequence=["lightcoral"],
              title="Nombre de prélèvements par mois")
fig2.update_layout(xaxis_title="Mois de prélèvement", yaxis_title="Nombre de personnes")

# region francaise
region_counts = V1_df['French_Region'].value_counts().reset_index()
region_counts.columns = ["Région", "Nombre"]
fig4 = px.bar(region_counts, x="Région", y="Nombre",
              color_discrete_sequence=["#2ca02c"],  # Vert
              title="Répartition par région française")
fig4.update_layout(xaxis_title="Région française",
                  yaxis_title="Nombre de personnes",
                  xaxis_tickangle=-45)

fig = make_subplots(rows=1, cols=3, subplot_titles=("Répartition des âges", "Prélèvements par mois", "Répartition par région française"),
                   shared_yaxes=False)

# Ajouter les sous-graphiques
for trace in fig1["data"]:
    fig.add_trace(trace, row=1, col=1)

for trace in fig2["data"]:
    fig.add_trace(trace, row=1, col=2)

for trace in fig4["data"]:
    fig.add_trace(trace, row=1, col=3)

# Mettre en page
fig.update_layout(
    height=500,
    width=1200,
    showlegend=False,
    margin=dict(r=50, l=50, b=100),  # Ajuster les marges pour les labels
    # Couleurs uniques pour chaque graphique
    colorway=["#1f77b4", "#ff7f0e", "#2ca02c"]
)


# Rotation des étiquettes sur l'axe x pour la région
fig.update_xaxes(tickangle=-45, row=1, col=3)

fig.show()

In [28]:
import plotly.express as px

# Calcul des pourcentages
allergy_counts = V1_df['Allergy_Present'].value_counts(normalize=True) * 100
df_allergy = pd.DataFrame({
    "Catégorie": allergy_counts.index.map({1: "Avec allergie", 0: "Sans allergie"}),
    "Pourcentage": allergy_counts.values
})

# Création du diagramme circulaire
fig = px.pie(
    df_allergy,
    names="Catégorie",
    values="Pourcentage",
    color="Catégorie",
    color_discrete_map={
        "Avec allergie": "#FF6B6B",  # Rouge clair
        "Sans allergie": "#4ECDC4"    # Turquoise
    },
    title="<b>Répartition des patients avec et sans allergie</b>",
    hole=0.3,  # Pour un donut chart (mettre 0 pour un pie chart classique)
    height=500,
    width=700
)


# Personnalisation du texte et des hover
fig.update_traces(
    textposition='inside',
    textinfo='percent+label',
    hovertemplate="<b>%{label}</b><br>Pourcentage: %{percent:.1%}",
    marker=dict(line=dict(color='#FFFFFF', width=1))
)

# Afficher le graphique
fig.show()

In [29]:
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots

# Liste des colonnes d'allergie
allergy_types = ["Respiratory_Allergy", "Food_Allergy", "Venom_Allergy"]
titles = ["Allergie Respiratoire", "Allergie Alimentaire", "Allergie au Venin"]

# Créer une figure avec sous-graphiques
fig = make_subplots(rows=1, cols=3,
                   specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]],
                   subplot_titles=titles)

# Couleurs constantes
colors = {'Avec': '#FF6B6B', 'Sans': '#4ECDC4'}

for i, col in enumerate(allergy_types, 1):
    # Calculer les pourcentages
    counts = V1_df[col].value_counts(normalize=True) * 100
    df = pd.DataFrame({
        'Status': ['Avec', 'Sans'],
        'Value': [counts.get(1, 0), counts.get(0, 0)]
    })

    # Ajouter chaque pie chart comme trace
    fig.add_trace(
        go.Pie(
            labels=df['Status'],
            values=df['Value'],
            name=titles[i-1],
            marker_colors=[colors['Avec'], colors['Sans']],
            hole=0.3,
            textinfo='percent+label',
            hovertemplate="<b>%{label}</b><br>Pourcentage: %{percent:.1%}",
            showlegend=False
        ),
        row=1, col=i
    )


# Afficher la figure
fig.show()

In [30]:
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Définition des colonnes et des libellés
allergy_columns = [
    "Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast"
]

clean_names = {
    "Type_of_Respiratory_Allergy_IGE_Pollen_Herb": "Pollen d'herbes",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree": "Pollen d'arbres",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals": "Poils d'animaux",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach": "Acariens/Cafards",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast": "Moississures/Lévures"
}

# Création de la figure avec sous-graphiques
fig = make_subplots(
    rows=2,
    cols=3,
    specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}],
           [{'type':'domain'}, {'type':'domain'}, None]],
    subplot_titles=[clean_names[col] for col in allergy_columns]
)

# Couleurs constantes
colors = {'Positif': '#E74C3C', 'Négatif': '#3498DB'}

for i, col in enumerate(allergy_columns):
    # Calcul des pourcentages
    counts = V1_df[col].value_counts(normalize=True) * 100
    df = pd.DataFrame({
        'Status': ['Positif', 'Négatif'],
        'Value': [counts.get(1, 0), counts.get(0, 0)]
    })

    # Position dans la grille
    row = i // 3 + 1
    col_pos = i % 3 + 1

    # Ajout du pie chart
    fig.add_trace(
        go.Pie(
            labels=df['Status'],
            values=df['Value'],
            name=clean_names[col],
            marker_colors=[colors['Positif'], colors['Négatif']],
            hole=0.4,
            textinfo='percent+label',
            hoverinfo='label+percent',
            showlegend=False  # Supprimer la légende individuelle
        ),
        row=row, col=col_pos
    )

# Mise en page
fig.update_layout(
    title_text="<b>Répartition des allergies respiratoires spécifiques</b>",
    height=600,
    width=900,
    title_x=0.5,
    uniformtext_minsize=12,
    uniformtext_mode='hide',
    legend_title_text="Résultat",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.15,
        xanchor="center",
        x=0.5
    )
)

# Ajouter une légende unique
fig.add_trace(
    go.Pie(
        labels=['Positif', 'Négatif'],
        values=[1, 1],  # Valeurs factices
        marker_colors=[colors['Positif'], colors['Négatif']],
        showlegend=True,
        hoverinfo='none',
        textinfo='none'
    ),
    row=2, col=2
)

fig.show()


In [31]:
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Définition des colonnes et libellés
food_allergy_columns = [
    "Type_of_Food_Allergy_Aromatics",
    "Type_of_Food_Allergy_Other",
    "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg",
    "Type_of_Food_Allergy_Fish",
    "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk",
    "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes",
    "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish",
    "Type_of_Food_Allergy_TPO",
    "Type_of_Food_Allergy_Tree_Nuts"
]

clean_names = {
    "Type_of_Food_Allergy_Aromatics": "Aromates",
    "Type_of_Food_Allergy_Other": "Autres",
    "Type_of_Food_Allergy_Cereals_&_Seeds": "Céréales & Graines",
    "Type_of_Food_Allergy_Egg": "Œufs",
    "Type_of_Food_Allergy_Fish": "Poisson",
    "Type_of_Food_Allergy_Fruits_and_Vegetables": "Fruits & Légumes",
    "Type_of_Food_Allergy_Mammalian_Milk": "Lait mammifère",
    "Type_of_Food_Allergy_Oral_Syndrom": "Syndrome oral",
    "Type_of_Food_Allergy_Other_Legumes": "Légumineuses",
    "Type_of_Food_Allergy_Peanut": "Arachide",
    "Type_of_Food_Allergy_Shellfish": "Crustacés",
    "Type_of_Food_Allergy_TPO": "TPO",
    "Type_of_Food_Allergy_Tree_Nuts": "Fruits à coque"
}

# Configuration de la grille (4 lignes x 4 colonnes)
fig = make_subplots(
    rows=4,
    cols=4,
    specs=[[{'type':'domain'}] * 4] * 4,
    subplot_titles=[clean_names[col] for col in food_allergy_columns]
)

# Couleurs constantes
colors = {'Positif': '#E74C3C', 'Négatif': '#3498DB'}

for i, col in enumerate(food_allergy_columns):
    # Calcul des pourcentages
    counts = V1_df[col].value_counts(normalize=True) * 100
    df = pd.DataFrame({
        'Status': ['Positif', 'Négatif'],
        'Value': [counts.get(1, 0), counts.get(0, 0)]
    })

    # Position dans la grille
    row = i // 4 + 1
    col_pos = i % 4 + 1

    # Ajout du pie chart
    fig.add_trace(
        go.Pie(
            labels=df['Status'],
            values=df['Value'],
            name=clean_names[col],
            marker_colors=[colors['Positif'], colors['Négatif']],
            hole=0.4,
            textinfo='percent+label',
            hoverinfo='label+percent',
            showlegend=False
        ),
        row=row, col=col_pos
    )

# Mise en page améliorée
fig.update_layout(
    title_text="<b>Répartition des allergies alimentaires par type</b>",
    height=900,
    width=1100,
    title_x=0.5,
    uniformtext_minsize=10,
    uniformtext_mode='hide',
    legend_title_text="Résultat",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5
    )
)

# Ajouter une légende unique
fig.add_trace(
    go.Pie(
        labels=['Positif', 'Négatif'],
        values=[1, 1],  # Valeurs factices
        marker_colors=[colors['Positif'], colors['Négatif']],
        showlegend=True,
        hoverinfo='none',
        textinfo='none'
    ),
    row=4, col=4
)

fig.show()


In [32]:
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Configuration
venom_data = {
    "Type_of_Venom_Allergy_ATCD_Venom": "Antécédents d'allergie",
    "Type_of_Venom_Allergy_IGE_Venom": "Tests IgE positifs"
}

colors = {'Positif': '#FF6B6B', 'Négatif': '#4ECDC4'}

# Création de la figure
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type':'domain'}, {'type':'domain'}]],
    horizontal_spacing=0.2
)

# Ajout des données
for i, (col, label) in enumerate(venom_data.items(), 1):
    counts = V1_df[col].value_counts(normalize=True) * 100
    fig.add_trace(go.Pie(
        labels=['Positif', 'Négatif'],
        values=[counts.get(1, 0), counts.get(0, 0)],
        name=label,
        marker_colors=[colors['Positif'], colors['Négatif']],
        hole=0.45,
        textinfo='percent+label',
        textfont_size=14,
        hovertemplate="<b>%{label}</b><br>%{percent:.1%}",
        showlegend=False
    ), 1, i)

# Mise en page avancée
fig.update_layout(
    height=450,
    width=850,
    margin=dict(t=100, b=50),  # Espace pour le titre
    annotations=[
        # Titre principal
        dict(
            text="<b>Prévalence des allergies au venin</b>",
            x=0.5, y=1.15,
            xref="paper", yref="paper",
            font_size=20,
            showarrow=False
        ),
        # Sous-titres
        *[dict(
            text=label,
            x=0.225 if i==1 else 0.775,
            y=1.05,
            xref="paper", yref="paper",
            font_size=14,
            showarrow=False
        ) for i, label in enumerate(venom_data.values(), 1)]]
)

# Ajout manuel de la légende
fig.add_annotation(
    x=0.5, y=-0.1,
    xref="paper", yref="paper",
    text="<span style='color:%s'>■</span> Positif <span style='color:%s'>■</span> Négatif" % (colors['Positif'], colors['Négatif']),
    showarrow=False,
    font_size=14
)

fig.show()

In [6]:
# Colonnes à conserver en plus des données entre "Act_d_1" et "Ves_v_5"
extra_columns = ["Patient_ID", "Chip_Code", "Allergy_Present", "Respiratory_Allergy", "Food_Allergy", "Venom_Allergy" ]

# Trouver les indices des colonnes "Act_d_1" et "Ves_v_5"
start_col = V1_df.columns.get_loc("Act_d_1")
end_col = V1_df.columns.get_loc("Ves_v_5")

# Sélectionner les colonnes spécifiques
selected_columns = extra_columns + list(V1_df.columns[start_col:end_col + 1])
V1_data = V1_df[selected_columns]

# Afficher les premières lignes du nouveau dataframe
print(V1_data.head())

# Enregistrer dans un nouveau fichier si besoin
#V1_data.to_excel("V1_data_filtered.xlsx", index=False, engine="openpyxl")

  Patient_ID Chip_Code  Allergy_Present  Respiratory_Allergy  Food_Allergy  \
0    XPW0007   XPW0007                0                    0             0   
1    XPW0011   XPW0011                0                    1             1   
2    XPW0013   XPW0013                0                    0             0   
3    XPW0017   XPW0017                0                    1             0   
4    XPW0018   XPW0018                0                    1             1   

   Venom_Allergy  Act_d_1  Act_d_2  Act_d_5  Act_d_8  ...  Pla_l_1  Pol_d_5  \
0              0     0.00     0.00      0.0     0.00  ...     0.00     0.14   
1              0     0.11     1.39      0.0     1.40  ...     0.00     0.00   
2              0     0.00     0.00      0.0     0.00  ...     0.00     0.00   
3              0     0.00     0.00      0.0     0.00  ...     0.00     0.00   
4              0     1.12     0.40      0.0     1.26  ...     9.08     0.00   

   Pru_p_1  Pru_p_3  Sal_k_1  Ses_i_1  Tri_a_14  Tri_a_1

In [48]:
print(V1_data.isna().sum())  # Nombre de NaN par colonne
print(V1_data.describe())  # Vérifie les valeurs aberrantes

Patient_ID             0
Chip_Code              0
Allergy_Present        0
Respiratory_Allergy    0
Food_Allergy           0
                      ..
Ses_i_1                0
Tri_a_14               0
Tri_a_19.0101          0
Tri_a_aA_TI            0
Ves_v_5                0
Length: 118, dtype: int64
       Allergy_Present  Respiratory_Allergy  Food_Allergy  Venom_Allergy  \
count      2351.000000          2351.000000    2351.00000    2351.000000   
mean          0.523182             0.809443       0.64866       0.078265   
std           0.499569             0.392824       0.47749       0.268645   
min           0.000000             0.000000       0.00000       0.000000   
25%           0.000000             1.000000       0.00000       0.000000   
50%           1.000000             1.000000       1.00000       0.000000   
75%           1.000000             1.000000       1.00000       0.000000   
max           1.000000             1.000000       1.00000       1.000000   

           Act

In [19]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 1. On garde les colonnes utiles
X = V1_data.drop(["Patient_ID", "Chip_Code", "Allergy_Present", "Food_Allergy", "Venom_Allergy", "Respiratory_Allergy"], axis=1)
y = V1_data["Allergy_Present"]

# 2. Supprimer les lignes avec NaN (optionnel si tu veux toutes les lignes : fais une imputation)
X_clean = X.dropna()
y_clean = y.loc[X_clean.index]
patient_ids = V1_data.loc[X_clean.index, "Patient_ID"]

# 3. Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# 4. PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# 5. Reconstruction d’un DataFrame propre
df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
df_pca["Patient_ID"] = patient_ids.values
df_pca["Allergy_Present"] = y_clean.values

import plotly.express as px

fig = px.scatter(
    df_pca,
    x="PC1",
    y="PC2",
    color=df_pca["Allergy_Present"].astype(str),
    hover_data=["Patient_ID"],
    title="Projection PCA des patients selon la composante respiratoire",
    labels={
        "PC1": "Première composante principale",
        "PC2": "Deuxième composante principale",
        "Allergy_Present": "Presance d'allergie"
    },
    color_discrete_map={"0": "blue", "1": "red"}
)

fig.update_layout(
    width=900,
    height=650,
    legend_title="Présence d'allergie",
)

fig.show()



In [20]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import plotly.express as px

# 1. On garde les colonnes utiles
X = V1_data.drop(["Patient_ID", "Chip_Code", "Allergy_Present", "Food_Allergy", "Venom_Allergy", "Respiratory_Allergy"], axis=1)
y = V1_data["Allergy_Present"]

# 2. Supprimer les lignes avec NaN
X_clean = X.dropna()
y_clean = y.loc[X_clean.index]
patient_ids = V1_data.loc[X_clean.index, "Patient_ID"]

# 3. Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# 4. PCA avec 3 composantes
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# 5. Reconstruction d’un DataFrame propre
df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2", "PC3"])
df_pca["Patient_ID"] = patient_ids.values
df_pca["Allergy_Present"] = y_clean.values

# 6. Visualisation interactive 3D
fig = px.scatter_3d(
    df_pca,
    x="PC1",
    y="PC2",
    z="PC3",
    color=df_pca["Allergy_Present"].astype(str),
    hover_data=["Patient_ID"],
    title="Projection PCA 3D des patients selon la composante respiratoire",
    labels={
        "PC1": "Composante 1",
        "PC2": "Composante 2",
        "PC3": "Composante 3",
        "Allergy_Present": "Présence d'allergie"
    },
    color_discrete_map={"0": "blue", "1": "red"}
)

fig.update_layout(
    width=1000,
    height=700,
    legend_title="Présence d'allergie",
)

fig.show()


In [17]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 1. On garde les colonnes utiles
X = V1_data.drop(["Patient_ID", "Chip_Code", "Allergy_Present", "Food_Allergy", "Venom_Allergy", "Respiratory_Allergy"], axis=1)
y = V1_data["Respiratory_Allergy"]

# 2. Supprimer les lignes avec NaN (optionnel si tu veux toutes les lignes : fais une imputation)
X_clean = X.dropna()
y_clean = y.loc[X_clean.index]
patient_ids = V1_data.loc[X_clean.index, "Patient_ID"]

# 3. Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# 4. PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# 5. Reconstruction d’un DataFrame propre
df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
df_pca["Patient_ID"] = patient_ids.values
df_pca["Respiratory_Allergy"] = y_clean.values

import plotly.express as px

fig = px.scatter(
    df_pca,
    x="PC1",
    y="PC2",
    color=df_pca["Respiratory_Allergy"].astype(str),
    hover_data=["Patient_ID"],
    title="Projection PCA des patients selon la composante respiratoire",
    labels={
        "PC1": "Première composante principale",
        "PC2": "Deuxième composante principale",
        "Respiratory_Allergy": "Allergie respiratoire"
    },
    color_discrete_map={"0": "blue", "1": "red"}
)

fig.update_layout(
    width=900,
    height=650,
    legend_title="Présence d'allergie",
)

fig.show()



In [18]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import plotly.express as px

# 1. On garde les colonnes utiles
X = V1_data.drop(["Patient_ID", "Chip_Code", "Allergy_Present", "Food_Allergy", "Venom_Allergy", "Respiratory_Allergy"], axis=1)
y = V1_data["Respiratory_Allergy"]

# 2. Supprimer les lignes avec NaN
X_clean = X.dropna()
y_clean = y.loc[X_clean.index]
patient_ids = V1_data.loc[X_clean.index, "Patient_ID"]

# 3. Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# 4. PCA avec 3 composantes
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# 5. Reconstruction d’un DataFrame propre
df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2", "PC3"])
df_pca["Patient_ID"] = patient_ids.values
df_pca["Respiratory_Allergy"] = y_clean.values

# 6. Visualisation interactive 3D
fig = px.scatter_3d(
    df_pca,
    x="PC1",
    y="PC2",
    z="PC3",
    color=df_pca["Respiratory_Allergy"].astype(str),
    hover_data=["Patient_ID"],
    title="Projection PCA 3D des patients selon la composante respiratoire",
    labels={
        "PC1": "Composante 1",
        "PC2": "Composante 2",
        "PC3": "Composante 3",
        "Respiratory_Allergy": "Allergie respiratoire"
    },
    color_discrete_map={"0": "blue", "1": "red"}
)

fig.update_layout(
    width=1000,
    height=700,
    legend_title="Présence d'allergie",
)

fig.show()


In [23]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

# 1. Préparation des données
X = V1_data.drop(["Patient_ID", "Chip_Code", "Allergy_Present", "Food_Allergy", "Venom_Allergy", "Respiratory_Allergy"], axis=1)
y = V1_data["Allergy_Present"]

# 2. Supprimer les lignes avec NaN
X_clean = X.dropna()
y_clean = y.loc[X_clean.index]

# 3. Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# 4. PCA (3 composantes)
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# 5. Définition des modèles
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(probability=True),
    "XGBoost": XGBClassifier(eval_metric="logloss", use_label_encoder=False, n_estimators=50)
}

# 6. Cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
results_auc = {model_name: [] for model_name in models}

# 7. Entraînement et évaluation
for train_index, test_index in kf.split(X_pca):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y_clean.iloc[train_index], y_clean.iloc[test_index]

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_proba)
        results_auc[model_name].append(auc)

# 8. Résultats
for model_name, aucs in results_auc.items():
    print(f"🔍 {model_name} - AUC moyen sur 10 folds : {np.mean(aucs):.4f}")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[19:54:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[19:54:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[19:54:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[19:54:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[19:54:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[19:54:24] WARNING: /workspace

🔍 Logistic Regression - AUC moyen sur 10 folds : 0.5355
🔍 Random Forest - AUC moyen sur 10 folds : 0.5120
🔍 SVM - AUC moyen sur 10 folds : 0.4989
🔍 XGBoost - AUC moyen sur 10 folds : 0.5219


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[19:54:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




In [24]:
import pandas as pd
import plotly.express as px

# On suppose que 'results_auc' est un dictionnaire contenant pour chaque modèle la liste des AUC obtenues par fold.
# Par exemple : results_auc = {"Logistic Regression": [0.85, 0.83, …], "Random Forest": [...], ...}

# Conversion en DataFrame pour Plotly
data = []
for model_name, auc_list in results_auc.items():
    for auc in auc_list:
        data.append({"Modèle": model_name, "AUC": auc})

df_auc = pd.DataFrame(data)

fig_box = px.box(df_auc, x="Modèle", y="AUC",
                 points="all",
                 title="Boxplot des AUC par modèle (10-fold CV)",
                 labels={"AUC": "AUC (Area Under ROC Curve)", "Modèle": "Modèle de Classification"})
fig_box.show()

In [26]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve

# Utiliser "Allergy_Present" comme cible
X = V1_data.drop(["Patient_ID", "Chip_Code", "Allergy_Present",
                  "Food_Allergy", "Venom_Allergy", "Respiratory_Allergy"], axis=1)
y = V1_data["Allergy_Present"]

# Supprimer les lignes avec NaN (ici, le jeu de données simulé n'en a pas)
X_clean = X.dropna()
y_clean = y.loc[X_clean.index]

# Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# PCA en 3 composantes
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# --- Définition des modèles ---
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(probability=True),
    "XGBoost": XGBClassifier(eval_metric="logloss", use_label_encoder=False, n_estimators=50)
}

# --- K-Fold Cross Validation ---
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Dictionnaires pour stocker les résultats
results_auc = {model_name: [] for model_name in models}
results_roc = {model_name: [] for model_name in models}

# Boucle de cross validation
for train_index, test_index in kf.split(X_pca):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y_clean.iloc[train_index], y_clean.iloc[test_index]

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_proba)
        fpr, tpr, _ = roc_curve(y_test, y_proba)

        results_auc[model_name].append(auc)
        results_roc[model_name].append((fpr, tpr))

# Affichage des AUC moyens dans la console
for model_name, auc_list in results_auc.items():
    print(f"{model_name} - AUC moyen sur 10 folds : {np.mean(auc_list):.4f}")

# --- 1. Plotly : Boxplots (courbes moustaches) des AUC ---
data = []
for model_name, auc_list in results_auc.items():
    for auc in auc_list:
        data.append({"Modèle": model_name, "AUC": auc})
df_auc = pd.DataFrame(data)

fig_box = px.box(df_auc, x="Modèle", y="AUC",
                 points="all",
                 title="Boxplot des AUC par modèle (10-fold CV)",
                 labels={"AUC": "AUC (Area Under ROC Curve)", "Modèle": "Modèle de Classification"})
fig_box.show()

# --- 2. Plotly : Courbes ROC moyennes ---
mean_fpr = np.linspace(0, 1, 100)
fig_roc = go.Figure()

for model_name, roc_list in results_roc.items():
    tprs = []
    for fpr, tpr in roc_list:
        interp_tpr = np.interp(mean_fpr, fpr, tpr)
        tprs.append(interp_tpr)
    mean_tpr = np.mean(tprs, axis=0)
    auc_mean = np.mean(results_auc[model_name])
    fig_roc.add_trace(go.Scatter(x=mean_fpr, y=mean_tpr, mode='lines',
                                 name=f"{model_name} (AUC = {auc_mean:.2f})"))

# Courbe de référence
fig_roc.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash'),
                             name="Hasard"))

fig_roc.update_layout(title="Courbes ROC moyennes (10-fold CV)",
                      xaxis_title="False Positive Rate",
                      yaxis_title="True Positive Rate")
fig_roc.show()

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[20:00:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[20:00:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[20:00:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[20:00:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[20:00:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[20:00:11] WARNING: /workspace

Logistic Regression - AUC moyen sur 10 folds : 0.5355
Random Forest - AUC moyen sur 10 folds : 0.5119
SVM - AUC moyen sur 10 folds : 0.4898
XGBoost - AUC moyen sur 10 folds : 0.5219


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[20:00:18] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




In [27]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve

# Utiliser "Allergy_Present" comme cible
X = V1_data.drop(["Patient_ID", "Chip_Code", "Allergy_Present",
                  "Food_Allergy", "Venom_Allergy", "Respiratory_Allergy"], axis=1)
y = V1_data["Respiratory_Allergy"]

# Supprimer les lignes avec NaN (ici, le jeu de données simulé n'en a pas)
X_clean = X.dropna()
y_clean = y.loc[X_clean.index]

# Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# PCA en 3 composantes
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# --- Définition des modèles ---
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(probability=True),
    "XGBoost": XGBClassifier(eval_metric="logloss", use_label_encoder=False, n_estimators=50)
}

# --- K-Fold Cross Validation ---
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Dictionnaires pour stocker les résultats
results_auc = {model_name: [] for model_name in models}
results_roc = {model_name: [] for model_name in models}

# Boucle de cross validation
for train_index, test_index in kf.split(X_pca):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y_clean.iloc[train_index], y_clean.iloc[test_index]

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_proba)
        fpr, tpr, _ = roc_curve(y_test, y_proba)

        results_auc[model_name].append(auc)
        results_roc[model_name].append((fpr, tpr))

# Affichage des AUC moyens dans la console
for model_name, auc_list in results_auc.items():
    print(f"{model_name} - AUC moyen sur 10 folds : {np.mean(auc_list):.4f}")

# --- 1. Plotly : Boxplots (courbes moustaches) des AUC ---
data = []
for model_name, auc_list in results_auc.items():
    for auc in auc_list:
        data.append({"Modèle": model_name, "AUC": auc})
df_auc = pd.DataFrame(data)

fig_box = px.box(df_auc, x="Modèle", y="AUC",
                 points="all",
                 title="Boxplot des AUC par modèle (10-fold CV)",
                 labels={"AUC": "AUC (Area Under ROC Curve)", "Modèle": "Modèle de Classification"})
fig_box.show()

# --- 2. Plotly : Courbes ROC moyennes ---
mean_fpr = np.linspace(0, 1, 100)
fig_roc = go.Figure()

for model_name, roc_list in results_roc.items():
    tprs = []
    for fpr, tpr in roc_list:
        interp_tpr = np.interp(mean_fpr, fpr, tpr)
        tprs.append(interp_tpr)
    mean_tpr = np.mean(tprs, axis=0)
    auc_mean = np.mean(results_auc[model_name])
    fig_roc.add_trace(go.Scatter(x=mean_fpr, y=mean_tpr, mode='lines',
                                 name=f"{model_name} (AUC = {auc_mean:.2f})"))

# Courbe de référence
fig_roc.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash'),
                             name="Hasard"))

fig_roc.update_layout(title="Courbes ROC moyennes (10-fold CV)",
                      xaxis_title="False Positive Rate",
                      yaxis_title="True Positive Rate")
fig_roc.show()

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:00:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:00:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:00:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:00:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:00:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:00:16] WARNING: /workspace

Logistic Regression - AUC moyen sur 10 folds : 0.9070
Random Forest - AUC moyen sur 10 folds : 0.8922
SVM - AUC moyen sur 10 folds : 0.8883
XGBoost - AUC moyen sur 10 folds : 0.8977


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:00:19] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




In [30]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve

# Utiliser "Allergy_Present" comme cible
X = V1_data.drop(["Patient_ID", "Chip_Code", "Allergy_Present",
                  "Food_Allergy", "Venom_Allergy", "Respiratory_Allergy"], axis=1)
y = V1_data["Venom_Allergy"]

# Supprimer les lignes avec NaN (ici, le jeu de données simulé n'en a pas)
X_clean = X.dropna()
y_clean = y.loc[X_clean.index]

# Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# PCA en 3 composantes
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# --- Définition des modèles ---
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(probability=True),
    "XGBoost": XGBClassifier(eval_metric="logloss", use_label_encoder=False, n_estimators=50)
}

# --- K-Fold Cross Validation ---
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Dictionnaires pour stocker les résultats
results_auc = {model_name: [] for model_name in models}
results_roc = {model_name: [] for model_name in models}

# Boucle de cross validation
for train_index, test_index in kf.split(X_pca):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y_clean.iloc[train_index], y_clean.iloc[test_index]

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_proba)
        fpr, tpr, _ = roc_curve(y_test, y_proba)

        results_auc[model_name].append(auc)
        results_roc[model_name].append((fpr, tpr))

# Affichage des AUC moyens dans la console
for model_name, auc_list in results_auc.items():
    print(f"{model_name} - AUC moyen sur 10 folds : {np.mean(auc_list):.4f}")

# --- 1. Plotly : Boxplots (courbes moustaches) des AUC ---
data = []
for model_name, auc_list in results_auc.items():
    for auc in auc_list:
        data.append({"Modèle": model_name, "AUC": auc})
df_auc = pd.DataFrame(data)

fig_box = px.box(df_auc, x="Modèle", y="AUC",
                 points="all",
                 title="Boxplot des AUC par modèle (10-fold CV)",
                 labels={"AUC": "AUC (Area Under ROC Curve)", "Modèle": "Modèle de Classification"})
fig_box.show()

# --- 2. Plotly : Courbes ROC moyennes ---
mean_fpr = np.linspace(0, 1, 100)
fig_roc = go.Figure()

for model_name, roc_list in results_roc.items():
    tprs = []
    for fpr, tpr in roc_list:
        interp_tpr = np.interp(mean_fpr, fpr, tpr)
        tprs.append(interp_tpr)
    mean_tpr = np.mean(tprs, axis=0)
    auc_mean = np.mean(results_auc[model_name])
    fig_roc.add_trace(go.Scatter(x=mean_fpr, y=mean_tpr, mode='lines',
                                 name=f"{model_name} (AUC = {auc_mean:.2f})"))

# Courbe de référence
fig_roc.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash'),
                             name="Hasard"))

fig_roc.update_layout(title="Courbes ROC moyennes (10-fold CV)",
                      xaxis_title="False Positive Rate",
                      yaxis_title="True Positive Rate")
fig_roc.show()

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:15:30] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:15:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:15:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:15:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:15:36] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:15:37] WARNING: /workspace

Logistic Regression - AUC moyen sur 10 folds : 0.6457
Random Forest - AUC moyen sur 10 folds : 0.6928
SVM - AUC moyen sur 10 folds : 0.4804
XGBoost - AUC moyen sur 10 folds : 0.6954


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:15:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




In [29]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve

# Utiliser "Allergy_Present" comme cible
X = V1_data.drop(["Patient_ID", "Chip_Code", "Allergy_Present",
                  "Food_Allergy", "Venom_Allergy", "Respiratory_Allergy"], axis=1)
y = V1_data["Venom_Allergy"]

# Supprimer les lignes avec NaN (ici, le jeu de données simulé n'en a pas)
X_clean = X.dropna()
y_clean = y.loc[X_clean.index]

# Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# PCA en 3 composantes
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# --- Définition des modèles ---
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(probability=True),
    "XGBoost": XGBClassifier(eval_metric="logloss", use_label_encoder=False, n_estimators=50)
}

# --- K-Fold Cross Validation ---
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Dictionnaires pour stocker les résultats
results_auc = {model_name: [] for model_name in models}
results_roc = {model_name: [] for model_name in models}

# Boucle de cross validation
for train_index, test_index in kf.split(X_pca):
    X_train, X_test = X_pca[train_index], X_pca[test_index]
    y_train, y_test = y_clean.iloc[train_index], y_clean.iloc[test_index]

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_proba)
        fpr, tpr, _ = roc_curve(y_test, y_proba)

        results_auc[model_name].append(auc)
        results_roc[model_name].append((fpr, tpr))

# Affichage des AUC moyens dans la console
for model_name, auc_list in results_auc.items():
    print(f"{model_name} - AUC moyen sur 10 folds : {np.mean(auc_list):.4f}")

# --- 1. Plotly : Boxplots (courbes moustaches) des AUC ---
data = []
for model_name, auc_list in results_auc.items():
    for auc in auc_list:
        data.append({"Modèle": model_name, "AUC": auc})
df_auc = pd.DataFrame(data)

fig_box = px.box(df_auc, x="Modèle", y="AUC",
                 points="all",
                 title="Boxplot des AUC par modèle (10-fold CV)",
                 labels={"AUC": "AUC (Area Under ROC Curve)", "Modèle": "Modèle de Classification"})
fig_box.show()

# --- 2. Plotly : Courbes ROC moyennes ---
mean_fpr = np.linspace(0, 1, 100)
fig_roc = go.Figure()

for model_name, roc_list in results_roc.items():
    tprs = []
    for fpr, tpr in roc_list:
        interp_tpr = np.interp(mean_fpr, fpr, tpr)
        tprs.append(interp_tpr)
    mean_tpr = np.mean(tprs, axis=0)
    auc_mean = np.mean(results_auc[model_name])
    fig_roc.add_trace(go.Scatter(x=mean_fpr, y=mean_tpr, mode='lines',
                                 name=f"{model_name} (AUC = {auc_mean:.2f})"))

# Courbe de référence
fig_roc.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash'),
                             name="Hasard"))

fig_roc.update_layout(title="Courbes ROC moyennes (10-fold CV)",
                      xaxis_title="False Positive Rate",
                      yaxis_title="True Positive Rate")
fig_roc.show()

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:03:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:03:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:04:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:04:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:04:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:04:03] WARNING: /workspace

Logistic Regression - AUC moyen sur 10 folds : 0.6457
Random Forest - AUC moyen sur 10 folds : 0.6921
SVM - AUC moyen sur 10 folds : 0.5031
XGBoost - AUC moyen sur 10 folds : 0.6954


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[21:04:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


